this is pretransformation file 

In [1]:
import os
import shutil
import pandas as pd

def move_images_to_structure(df, base_dir):
    """
    Move images from the old structure to the new one based on `study_id` and `series_id`.
    """
    for study_id, series_id, image_path in zip(df['study_id'], df['series_id'], df['image_path']):
        # Extract study and series directories
        study_folder = os.path.join(base_dir, str(study_id))
        series_folder = os.path.join(study_folder, str(series_id))
        
        # Create directories if they don't exist
        os.makedirs(series_folder, exist_ok=True)
        
        # Get image filename
        image_filename = os.path.basename(image_path)
        
        # Define the new image path
        dest_image_path = os.path.join(series_folder, image_filename)
        
        # Check if the image exists before moving
        if os.path.exists(image_path):
            shutil.move(image_path, dest_image_path)
            print(f"Moved: {image_path} -> {dest_image_path}")
        else:
            print(f"Image not found: {image_path}")

# Example dataframe
data = {
    'study_id': [4003253, 4003253, 4003253, 4003253, 4003253],
    'severity': [1.0, 1.0, 1.0, 1.0, 1.0],
    'condition': ['left neural foraminal narrowing']*5,
    'level': ['l5/s1']*5,
    'series_id': [702807833, 702807833, 702807833, 702807833, 702807833],
    'x': [197.100569]*5,
    'y': [289.457306]*5,
    'image_path': [
        'data/train_images_origin/4003253/702807833/1.dcm',
        'data/train_images_origin/4003253/702807833/2.dcm',
        'data/train_images_origin/4003253/702807833/3.dcm',
        'data/train_images_origin/4003253/702807833/4.dcm',
        'data/train_images_origin/4003253/702807833/5.dcm'
    ],
    'missing_image': [False]*5
}

df = pd.DataFrame(data)

# Base directory where the images will be moved
base_dir = 'colab'

# Call the function to move images
move_images_to_structure(df, base_dir)


Moved: data/train_images_origin/4003253/702807833/1.dcm -> colab\4003253\702807833\1.dcm
Moved: data/train_images_origin/4003253/702807833/2.dcm -> colab\4003253\702807833\2.dcm
Moved: data/train_images_origin/4003253/702807833/3.dcm -> colab\4003253\702807833\3.dcm
Moved: data/train_images_origin/4003253/702807833/4.dcm -> colab\4003253\702807833\4.dcm
Moved: data/train_images_origin/4003253/702807833/5.dcm -> colab\4003253\702807833\5.dcm


In [ ]:
import os
import cv2
import glob
import time
from torchvision import transforms
from tqdm import tqdm

# Define the transform with augmentation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(10),       # Randomly rotate the image by ±10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color properties
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Path to your images and where to save augmented images
input_folder = 'path/to/original/images'
output_folder = 'path/to/augmented/images'

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Start timing
start_time = time.time()

# Process and save images
for image_path in tqdm(glob.glob(os.path.join(input_folder, '*.*'))):  # Adjust for your image file types
    image = cv2.imread(image_path)  # Read the image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB if using OpenCV

    # Apply transformations
    image_tensor = transform(image)

    # Convert back to numpy array and save
    image_transformed = image_tensor.permute(1, 2, 0).numpy()  # Change back to HxWxC
    image_transformed = (image_transformed * 255).astype('uint8')  # Scale back to uint8

    # Save the transformed image
    output_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_path, cv2.cvtColor(image_transformed, cv2.COLOR_RGB2BGR))  # Convert back to BGR for OpenCV

# End timing
end_time = time.time()
total_time = end_time - start_time

print(f"Pre-transformation complete! Total time: {total_time:.2f} seconds.")
print(f"Average time per image: {total_time / len(glob.glob(os.path.join(input_folder, '*.*'))):.4f} seconds.")


In [7]:
import pandas as pd
import sys
from pathlib import Path
import importlib

data_path = Path('C:/Users/HP1/Desktop/Spiced/child-mind-institute-problematic-internet-use/series_train.parquet/id=00f332d1')
 

# Read the Parquet file
df = pd.read_parquet(data_path  /"part-0.parquet")

In [8]:
def keep_persons(dataframes, study_ids_to_keep, all_studies=False):
    if all_studies:
        return dataframes  # Return all DataFrames if all_studies is True
    
    filtered_dataframes = []
    for df in dataframes:
        # Filter the DataFrame based on study_ids_to_keep
        filtered_df = df[df['study_id'].isin(study_ids_to_keep)]
        filtered_dataframes.append(filtered_df)  # Append the filtered DataFrame to the list

    return filtered_dataframes 

 
import pandas as pd

# Sample DataFrame
data = {
    'study_id': [4003253, 4003254, 4003253, 4003255],
    'patient_name': ['Alice', 'Bob', 'Charlie', 'David']
}

df_image_paths = pd.DataFrame(data)

# Using the corrected function
study_ids_to_keep = [4003253]
dataframes = [df_image_paths]

filtered_dataframes = keep_persons(dataframes, study_ids_to_keep, all_studies=False)

# Print the filtered DataFrames
for filtered_df in filtered_dataframes:
    print(filtered_df)


   study_id patient_name
0   4003253        Alice
2   4003253      Charlie


In [9]:
import pandas as pd
import numpy as np

# Define the df_train DataFrame
df_train = pd.DataFrame({
    "site_id": [2, 2, 2, 2, 2],
    "patient_id": [10006, 10006, 10006, 10006, 10011],
    "image_id": [462822612, 1459541791, 1864590858, 1874946579, 220375232],
    "laterality": ["L", "L", "R", "R", "L"],
    "view": ["CC", "MLO", "MLO", "CC", "CC"],
    "age": [61.0, 61.0, 61.0, 61.0, 55.0],
    "cancer": [0, 0, 0, 0, 0],
    "biopsy": [0, 0, 0, 0, 0],
    "invasive": [0, 0, 0, 0, 0],
    "BIRADS": [np.nan, np.nan, np.nan, np.nan, 0.0],
    "implant": [0, 0, 0, 0, 0],
    "density": [np.nan, np.nan, np.nan, np.nan, np.nan],
    "machine_id": [29, 29, 29, 29, 21],
    "difficult_negative_case": [False, False, False, False, True]
})

df_train

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True


In [10]:
# Code to create the 'data' DataFrame with the 'class' column
data = pd.DataFrame(
    np.concatenate([
        ['Total'] * len(df_train),  # Label 'Total' for each row in df_train
        ['Malignant Cancer'] * len(df_train[df_train['cancer'] == 1]),  # Label 'Malignant Cancer' for rows with cancer == 1
        ['Invasive Cancer'] * len(df_train[(df_train['cancer'] == 1) & (df_train['invasive'] == 1)])  # Label 'Invasive Cancer' for rows with cancer == 1 and invasive == 1
    ]),
    columns=["class"]
)

print(data)

   class
0  Total
1  Total
2  Total
3  Total
4  Total


In [11]:
# The not-malignant cancer cases were limited to biopsy cases.
DF_train = df_train[df_train['biopsy'] == 1].reset_index(drop = True)
DF_train.head()

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
